In [ ]:
# TODO pulisci questo file
# https://arxiv.org/pdf/1803.09820
# https://machinelearningmastery.com/learning-rate-for-deep-learning-neural-networks/

In [1]:
import os

from torch.utils.data import DataLoader

os.environ['KERAS_BACKEND'] = "torch"

## Regularization
>We hope to learn vector representations of the most representative aspects for a review dataset. <br>
However, the aspect embedding matrix T may suffer from redundancy problems during training. [...] <br>
> The regularization term encourages orthogonality among the rows of the aspect embedding matrix T and penalizes redundancy between different aspect vectors <br>
> ~ Ruidan

We use an Orthogonal Regulizer definition of the method can be found here: https://paperswithcode.com/method/orthogonal-regularization. <br/>
For the code we use the default implementation provided by Keras (https://keras.io/api/layers/regularizers/)

## Hands on first attempt:


In [2]:
from core.dataset import PositiveNegativeCommentGeneratorDataset, TokenizedDataset
from core.train import AbaeModelManager, AbaeModelConfiguration

corpus = "../output/dataset/pre-processed/200k.preprocessed.csv"
config = AbaeModelConfiguration(corpus_file=corpus, model_name=f"hands_on")

print(f"Running on default config:\n {config}")

# Without any hp tuning we just try and see how it goes.
manager = AbaeModelManager(config)
train_dataset = PositiveNegativeCommentGeneratorDataset(
    vocabulary=manager.embedding_model.vocabulary(),
    csv_dataset_path=config.corpus_file, negative_size=config.negative_sample_size
)

Running on default config:
 AbaeModelConfiguration(corpus_file='../output/dataset/pre-processed/200k.preprocessed.csv', model_name='hands_on', aspect_size=16, max_vocab_size=None, embedding_size=200, learning_rate=0.01, decay_rate=0.95, momentum=0.9, max_sequence_length=80, negative_sample_size=15, batch_size=64, epochs=10, output_path='./output')


Pandas Apply:   0%|          | 0/210167 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/210167 [00:00<?, ?it/s]

INFO:gensim.utils:loading Word2Vec object from output\hands_on\embeddings.keras
DEBUG:smart_open.smart_open_lib:{'uri': 'output\\hands_on\\embeddings.keras', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
INFO:gensim.utils:loading wv recursively from output\hands_on\embeddings.keras.wv.* with mmap=None
INFO:gensim.utils:setting ignored attribute cum_table to None
INFO:gensim.utils:Word2Vec lifecycle event {'fname': 'output\\hands_on\\embeddings.keras', 'datetime': '2025-01-22T20:03:23.636756', 'gensim': '4.3.3', 'python': '3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.26100-SP0', 'event': 'loaded'}


Loading dataset from file: ../output/dataset/pre-processed/200k.preprocessed.csv
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/210167 [00:00<?, ?it/s]

Max sequence length calculation in progress...
Max sequence length is:  431 . The limit is set to 80 tokens.
We loose information on 169 points.This is 0.08041224359675876% of the dataset.
Padding sequences to length (80).


In [ ]:
manager.run_train_process(train_dataset)

### Evaluation

In [25]:
# THIS IS OKAY!
from gensim import corpora
# See if another coherence metric might be better.
from core.evaluation import coherence_model_generation, normalize, get_aspect_top_k_words
import json
from core.dataset import PositiveNegativeCommentGeneratorDataset, TokenizedDataset
from core.train import AbaeModelManager, AbaeModelConfiguration

c_file = "../output/dataset/pre-processed/tuning.preprocessed.csv"
# Load aspects
config = AbaeModelConfiguration(corpus_file=corpus, model_name=f"hands_on")

manager = AbaeModelManager(config)
iteration_model = manager.get_model()

# word_emb = normalize(iteration_model.get_layer('word_embedding').weights[0].value.data)
# aspect_embeddings = normalize(iteration_model.get_layer('aspect_embedding').w)
word_emb = iteration_model.get_layer('word_embedding').weights[0].value.data
aspect_embeddings = iteration_model.get_layer('aspect_embedding').w

inv_vocab = manager.embedding_model.model.wv.index_to_key

aspects_top_k_words = [get_aspect_top_k_words(a, word_emb, inv_vocab, top_k=3) for a in aspect_embeddings]
aspect_words = [[word[0] for word in aspect] for aspect in aspects_top_k_words]  # Remap

ds = TokenizedDataset(c_file, manager.embedding_model.vocabulary())
dictionary = corpora.Dictionary(ds.text_ds.apply(lambda x: x.split(' ')).to_list())

_, m = coherence_model_generation(aspect_words, ds.text_ds.apply(lambda x: x.split(' ')), dictionary, topn=3)

m.get_coherence()

Pandas Apply:   0%|          | 0/210167 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/210167 [00:00<?, ?it/s]

INFO:gensim.utils:loading Word2Vec object from output\hands_on\embeddings.keras
DEBUG:smart_open.smart_open_lib:{'uri': 'output\\hands_on\\embeddings.keras', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
INFO:gensim.utils:loading wv recursively from output\hands_on\embeddings.keras.wv.* with mmap=None
INFO:gensim.utils:setting ignored attribute cum_table to None
INFO:gensim.utils:Word2Vec lifecycle event {'fname': 'output\\hands_on\\embeddings.keras', 'datetime': '2025-01-22T19:45:26.770796', 'gensim': '4.3.3', 'python': '3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.26100-SP0', 'event': 'loaded'}



Given aspect most representative words are:
Word:  clue (3.803128480911255)
Word:  answer (3.709832191467285)
Word:  question (3.5336668491363525)

Given aspect most representative words are:
Word:  story (6.793181419372559)
Word:  theme (6.318557262420654)
Word:  narrative (5.501640796661377)

Given aspect most representative words are:
Word:  play (8.043310165405273)
Word:  wife (5.442655563354492)
Word:  son (5.2307329177856445)

Given aspect most representative words are:
Word:  coal (5.949664115905762)
Word:  transport (5.541802883148193)
Word:  oil (5.515894412994385)

Given aspect most representative words are:
Word:  escape (6.336397171020508)
Word:  murder (5.624995231628418)
Word:  drink (5.465612888336182)

Given aspect most representative words are:
Word:  faq (7.361208438873291)
Word:  page (7.13925838470459)
Word:  instruction (6.736773490905762)

Given aspect most representative words are:
Word:  star::star::star::star::star::star::star::star::nostar::nostar (6.19304656

Pandas Apply:   0%|          | 0/83855 [00:00<?, ?it/s]

Max sequence length calculation in progress...
Max sequence length is:  274 . The limit is set to 80 tokens.
We loose information on 54 points.This is 0.06439687555900066% of the dataset.
Padding sequences to length (80).


INFO:gensim.corpora.dictionary:adding document #0 to Dictionary<0 unique tokens: []>
INFO:gensim.corpora.dictionary:adding document #10000 to Dictionary<10158 unique tokens: ['abstract', 'book', 'game', 'great', 'lose']...>
INFO:gensim.corpora.dictionary:adding document #20000 to Dictionary<14416 unique tokens: ['abstract', 'book', 'game', 'great', 'lose']...>
INFO:gensim.corpora.dictionary:adding document #30000 to Dictionary<17607 unique tokens: ['abstract', 'book', 'game', 'great', 'lose']...>
INFO:gensim.corpora.dictionary:adding document #40000 to Dictionary<20241 unique tokens: ['abstract', 'book', 'game', 'great', 'lose']...>
INFO:gensim.corpora.dictionary:adding document #50000 to Dictionary<22871 unique tokens: ['abstract', 'book', 'game', 'great', 'lose']...>
INFO:gensim.corpora.dictionary:adding document #60000 to Dictionary<25140 unique tokens: ['abstract', 'book', 'game', 'great', 'lose']...>
INFO:gensim.corpora.dictionary:adding document #70000 to Dictionary<26956 unique 

0.027082540569888416

## Aspect Embedding Size
The aspect embedding size is what will be inferring aspects. It is closest to representative words (?). <br />
We have to identify 7 actual aspects (luck, bookkeeping, downtime...) but that does not mean our matrix should be limited to rows only! <br>

For the first try we setup the aspect_size:
>The optimal number of rows is problem-dependent, so it’s crucial to: <br/>
> Start with a heuristic: Begin with 2–3x the number of aspects.

For **aspect extraction**, which involves identifying key aspects or topics in text, the best early stopping method depends on your approach:

### 1. Embedding-based Methods (e.g., Clustering Embeddings)
- **Silhouette Score**: Measure the separation and compactness of clusters. Stop when the score stabilizes.
- **Inertia/Distortion**: Track the sum of squared distances within clusters and stop when improvement flattens.
- **Centroid Movement**: Stop when the change in cluster centroids across iterations is minimal.

### 2. Topic Modeling (e.g., LDA)
- **Perplexity**: Monitor the perplexity on a held-out dataset and stop when it stops decreasing significantly.
- **Coherence Score**: Measure the semantic consistency of extracted topics and stop when it stabilizes.

### 3. Autoencoder-based Aspect Extraction
- **Reconstruction Loss**: Stop training when the validation reconstruction error no longer improves.

### 4. Qualitative Evaluation (if feasible)
- Periodically inspect extracted aspects for meaningfulness and diversity to decide on stopping.

For **aspect extraction**, combining an automated metric (like coherence score or silhouette score) with manual inspection often yields the best results.


## Hyperparameters Tuning
To tune our parameters we use a filtered version of the 50k ds. <br>
We filter out rows that can be found on the 200k ds.

In [ ]:
import pandas as pd

# This is based on the idea that our dataset are generated with different seeds else it won't work
large = pd.read_csv("../output/dataset/pre-processed/200k.preprocessed.csv")
small = pd.read_csv("../output/dataset/pre-processed/100k.preprocessed.csv")
tuning_set = small[~small["comments"].isin(large["comments"])]

tuning_set.to_csv("../output/dataset/pre-processed/tuning.preprocessed.csv", index=False)

> The main goal of ABAE is to extract interpretable and meaningful aspects, which makes coherence the more aligned metric.<br> Reconstruction error might help guide training but doesn’t guarantee that the extracted aspects are semantically useful.

In [ ]:
from core.hp_tuning import ABAERandomHyperparametersSelectionWrapper, HyperparameterTuningManager

configurations = 4  # We try 15 different configurations
corpus_file = "../output/dataset/pre-processed/tuning.preprocessed.csv"

print(f"Starting procedure. We try a total of {configurations}")
hp_wrapper = ABAERandomHyperparametersSelectionWrapper.create()
hp_tuning_manager = HyperparameterTuningManager(hp_wrapper, corpus_file, "./output")

In [ ]:
hp_tuning_manager(different_configurations=configurations, repeat=3)
# The output of this cell is very long therefore I deleted it

In [37]:
from pathlib import Path
import json
import pandas as pd
import numpy as np

dataframe = pd.DataFrame(
    columns=[
        'name', 'mean_loss', 'mean_coherence', 'aspect_size', 'embedding_size', 'epochs', 'batch_size', 'learning_rate',
        'decay_rate', 'momentum', 'negative_sample_size'
    ]
)

i = 0
# Inspect results we stored for process. todo: Quando finisci salvalo come file da "tenere" in repo cosi che chi controlla lo ha gia.
for element in Path("./output").iterdir():

    # Search only for the tuning process runs.
    if element.is_dir() and element.name.startswith("tuning"):
        run_result = json.load(open(str(element) + "/run_results.json"))
        # We want the best configurations.

        obj = {
            'name': element.name,
            'mean_loss': np.mean(run_result["evaluation_loss"]),
            'mean_coherence': np.mean(run_result["coherence"])
        }
        temp = pd.DataFrame(run_result['params'] | obj, index=[i])
        dataframe = pd.concat([temp, dataframe])
        i += 1

In [14]:
ds.text_ds.apply(lambda x: x.split(' '))

0        [abstract, game, tend, lose, point, book, grea...
1        [challenging, interesting, game, bet, future, ...
2        [turn, somewhat, cut, dry, little, turn, prese...
3        [rate, highly, <num>, component, plastic, <num...
4        [tile, player, allow, merging, hotel, possibly...
                               ...                        
83850    [✔, ️what, like, >, >, benefit, benefit, oppon...
83851    [loki, card, add, tactic, simple, draw, card, ...
83852    [❌, dislike, >, >, straightforward, script, fe...
83853    [<game_name>, desert, space, confusingly, simi...
83854    [check, need, head, head, racing, game, play, ...
Name: comments, Length: 83855, dtype: object

In [2]:
# THIS IS OKAY!

from gensim import corpora
# See if another coherence metric might be better.
from core.evaluation import coherence_model_generation, normalize, get_aspect_top_k_words
import json
from core.dataset import PositiveNegativeCommentGeneratorDataset, TokenizedDataset
from core.train import AbaeModelManager, AbaeModelConfiguration

model_name = 'tuning_7128da5c-04c0-4e91-abf2-462f9646b48d'
c_file = "../output/dataset/pre-processed/tuning.preprocessed.csv"
# Load aspects
parameters = json.load(open(f"./output/{model_name}/run_results.json"))['params']
config = AbaeModelConfiguration(corpus_file=c_file, model_name=model_name, **parameters)

manager = AbaeModelManager(config)
iteration_model = manager.get_model()

word_emb = normalize(iteration_model.get_layer('word_embedding').weights[0].value.data)
aspect_embeddings = normalize(iteration_model.get_layer('aspect_embedding').w)

# word_emb = iteration_model.get_layer('word_embedding').weights[0].value.data
# aspect_embeddings = iteration_model.get_layer('aspect_embedding').w

inv_vocab = manager.embedding_model.model.wv.index_to_key

aspects_top_k_words = [get_aspect_top_k_words(a, word_emb, inv_vocab, top_k=20) for a in aspect_embeddings]
aspect_words = [[word[0] for word in aspect] for aspect in aspects_top_k_words]  # Remap

ds = TokenizedDataset(c_file, manager.embedding_model.vocabulary())
dictionary = corpora.Dictionary(ds.text_ds.apply(lambda x: x.split(' ')).to_list())

_, m = coherence_model_generation(aspect_words, ds.text_ds.apply(lambda x: x.split(' ')), dictionary, topn=10)

m.get_coherence()

Pandas Apply:   0%|          | 0/83855 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/83855 [00:00<?, ?it/s]

INFO:gensim.utils:loading Word2Vec object from output\tuning_7128da5c-04c0-4e91-abf2-462f9646b48d\embeddings.keras
DEBUG:smart_open.smart_open_lib:{'uri': 'output\\tuning_7128da5c-04c0-4e91-abf2-462f9646b48d\\embeddings.keras', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
INFO:gensim.utils:loading wv recursively from output\tuning_7128da5c-04c0-4e91-abf2-462f9646b48d\embeddings.keras.wv.* with mmap=None
INFO:gensim.utils:setting ignored attribute cum_table to None
INFO:gensim.utils:Word2Vec lifecycle event {'fname': 'output\\tuning_7128da5c-04c0-4e91-abf2-462f9646b48d\\embeddings.keras', 'datetime': '2025-01-22T20:00:06.299579', 'gensim': '4.3.3', 'python': '3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.26100-SP0', 'event': 'loaded'}



Given aspect most representative words are:
Word:  best (0.3917645215988159)
Word:  geek (0.3776091933250427)
Word:  usa (0.36337709426879883)
Word:  | (0.3593183159828186)
Word:  gateway (0.3585318326950073)
Word:  resistance (0.34926456212997437)
Word:  adult (0.3304588794708252)
Word:  reiner (0.3275870084762573)
Word:  alike (0.3251059055328369)
Word:  king (0.3236229717731476)
Word:  friendly (0.31790152192115784)
Word:  club (0.31589022278785706)
Word:  social (0.31512516736984253)
Word:  2–4 (0.31502899527549744)
Word:  unplaye (0.3142813742160797)
Word:  des (0.31028759479522705)
Word:  expansions (0.310058057308197)
Word:  players (0.30858248472213745)
Word:  john (0.3030758500099182)
Word:  g (0.3012177646160126)

Given aspect most representative words are:
Word:  history (0.5361068844795227)
Word:  historical (0.5357145071029663)
Word:  recreate (0.496360182762146)
Word:  air (0.4897400140762329)
Word:  realistic (0.4780125021934509)
Word:  historically (0.47273367643356323

Pandas Apply:   0%|          | 0/83855 [00:00<?, ?it/s]

Max sequence length calculation in progress...
Max sequence length is:  274 . The limit is set to 80 tokens.
We loose information on 54 points.This is 0.06439687555900066% of the dataset.
Padding sequences to length (80).


INFO:gensim.corpora.dictionary:adding document #0 to Dictionary<0 unique tokens: []>
INFO:gensim.corpora.dictionary:adding document #10000 to Dictionary<10158 unique tokens: ['abstract', 'book', 'game', 'great', 'lose']...>
INFO:gensim.corpora.dictionary:adding document #20000 to Dictionary<14416 unique tokens: ['abstract', 'book', 'game', 'great', 'lose']...>
INFO:gensim.corpora.dictionary:adding document #30000 to Dictionary<17607 unique tokens: ['abstract', 'book', 'game', 'great', 'lose']...>
INFO:gensim.corpora.dictionary:adding document #40000 to Dictionary<20241 unique tokens: ['abstract', 'book', 'game', 'great', 'lose']...>
INFO:gensim.corpora.dictionary:adding document #50000 to Dictionary<22871 unique tokens: ['abstract', 'book', 'game', 'great', 'lose']...>
INFO:gensim.corpora.dictionary:adding document #60000 to Dictionary<25140 unique tokens: ['abstract', 'book', 'game', 'great', 'lose']...>
INFO:gensim.corpora.dictionary:adding document #70000 to Dictionary<26956 unique 

KeyboardInterrupt: 

In [5]:
manager.embedding_model.model.wv

In [4]:
ds[1]

([array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    1, 1437,    1, 1450, 1440,  276,
            1, 1440,    1]),
  []],
 0)

In [3]:
from core.dataset import EmbeddingsDataset, SimpleEmbeddingsDataset

iteration_model = manager.get_model()
custom_review = [
    'This game is hard to play but fun! Luck also plays a huge role!',
    'My brother takes forever to decide what to do',
    'Zero luck it is all skill'
]
# todo rebuild a prediciton model senza negative predict. Devo ristrutturare tutto1
ds = SimpleEmbeddingsDataset(custom_review, manager.embedding_model.model)
res = iteration_model.predict(DataLoader(ds, batch_size=32))

ValueError: Layer "functional" expects 2 input(s), but it received 1 input tensors. Inputs received: [tensor([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    1,    0,    1,   79, 1440,    3,
            1,    1,    1,    1, 2662,    1,  242,    1],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    

In [ ]:
m.get_coherence_per_topic()

In [9]:
len(ds.dataset.to_list())
list(filter(lambda x: x.any() is None or len(x) < 1, ds.dataset.to_list()))

[]

# Focus on learning rate

In [ ]:
# We fix other params and now focus entirely on lr.
# We have already a "promising" range defined.
# We look in that space so we redefine lr on ABAERandomHyperparametersSelectionWrapper

## Best found model training:

## See if the Hp tuning really improved upon our results:
We used SGD anda learned its parameters under the assumption that we would do better. <br>
Let's see if it really is the case, or we just wasted time.

For comparison we use Adam that has the advantage of being robust enough without parameter scouting.

In [ ]:
#todo

## Test accuracy on small test sample we filled out

### Test set definition

In [ ]:
from core.pre_processing import PreProcessingService
from core.pre_processing import DatasetGeneration

# We take around 1k records that were not seen yet from the model and label them by hand.
dataset = pd.read_csv("../data/corpus.csv")
# Handle game names. Or not? I don't need the full pipeline I guess. todo
game_names = pd.read_csv("../resources/2024-08-18.csv")['Name']
game_names = pd.concat([game_names, pd.Series(["Quick", "Catan"])], ignore_index=True)
document_game_names = game_names.swifter.apply(lambda x: nlp(x)).tolist()

pipeline = PreProcessingService.full_pipeline(document_game_names, "../data/processed-dataset/full")

# Extract 1k from dataset that are not in 200k
train_ds = pd.read_csv("../output/dataset/pre-processed/200k.preprocessed.csv")

# Take top 2k. (We will select some good ones and reduce the number to 1k)
test_set = dataset[~dataset["comments"].isin(train_ds["comments"])]

We have to use labels:

In [ ]:
labels = {
    '0': "Luck/Alea",
    '1': 'Bookkeeping',
    '2': 'Downtime',
    '3': 'Interaction',
    '4': 'Bash',
    '5': 'Complicated/Complex',  # I could watch weight to see if there is a ratio relation.
    '6': 'Misc'
}